Заполните данные о необходимых переменных:

In [133]:
Today = '2024-05-31'
Mounth = 'May'

Print_ARAP = False
print_manual_map = True
print_counterclaims = True
excel_tofolder_on_Z = False

import os
from datetime import date
import pandas as pd
import numpy as np
import sys

sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

CCY_tech_dict = {'RUC':'CHF','RUE':'EUR','RUE2':'EUR','RUE3':'EUR','RUE5':'EUR','RUG':'GBP','RUJ':'JPY','RUK':'KZT','RUM':'MDL','RUU':'USD','RUU1':'USD','RUU1R':'USD','RUU2':'USD','RUU25':'USD','RUU26':'USD','RUU3':'USD','RUU5':'USD','RUY':'CNY','RUK5':'KZT','RUY3':'CNY','RUY5':'CNY'}

Открываю следующие файлы:

In [2]:
query = f'''select 
    "customerName","internalExternalEN","orgName", "companyCode", "dueDate", "presentAmountUSD", "dealNumber", "holding","currencyCode", "sourceSystem",
CASE
        WHEN "bscCode" in ('130','135','BSC130','BSC135') 
        THEN 'Trade receivables'
        WHEN "bscCode" in ('160','165','BSC160','BSC165') 
        THEN 'Advances to suppliers'
        WHEN "bscCode" in ('170','175','BSC170','BSC175')
        THEN 'Capital advances'
        WHEN "bscCode" in ('140','145','BSC140','BSC145') 
        THEN 'Other receivables'
        WHEN "bscCode" in ('330','BSC330')
        THEN 'Provisions'
        WHEN "bscCode" in ('300','305','BSC300','BSC305') 
        THEN 'Trade payables'
        WHEN "bscCode" in ('310','315','BSC310','BSC315') 
        THEN 'Other payables'
        WHEN "bscCode" in ('360','365','BSC360','BSC365') 
        THEN 'Advances received'
        WHEN "bscCode" in ('350','355','BSC350','BSC355') 
        THEN 'Loans received'
        WHEN "bscCode" in ('210','215','BSC210','BSC215') 
        THEN 'Loans issued'
        WHEN "bscCode" in ('340','BSC340') 
        THEN 'Tax'
        WHEN "bscCode" in ('410','BSC410') 
        THEN 'Pension'
        ELSE 'Unknown'
    END AS "accountPurpose"
from RISKACCESS."unifiedARReport"
where 
    "sourceSystem" in ('OEBS12_EU','ETL document loader','SAP_HRG') and
    "rowType" = 'M' and
    "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD')  
union all select
     "customerName","internalExternalEN","orgName", "companyCode", "dueDate", "presentAmountUSD", "dealNumber", "holding","currencyCode", "sourceSystem",
CASE
        WHEN ("glAccount" like '62%' or "glAccount" IN (7699200310, 7699100210,4601000000, 4611000000, 1501010009, 1501010019, 1501010109, 1501010119, 2201010000,
        2201010008, 2201010009, 2201010010, 2201010018, 2201010019, 2201011000, 2201011008, 2201011009, 2201011010, 2201011018, 2201011019, 2201010098, 2201010099,
        2201020000, 2201020008, 2201030000, 2201030008, 2201010090, 2201010190, 2110000000, 2110000900, 2130000000, 2130000900, 2151000000, 2151000900, 2170000000,
        2170000900, 1210000000, 1210000900, 1260000000, 2181000000, 2181000900))
        THEN 'Trade receivables'
        WHEN "glAccount" IN (6002770900, 6002090900, 6002020000, 6002020099, 6002080000, 6002090000, 6002050000, 6002020900, 6002070000, 6002070099, 6002770000,
        6002050900, 6002070900, 6099000215, 6099000224, 6099000212, 6099000217, 6099000219, 6099000619, 6099000120, 6099000123, 6099000820, 6099000430, 6099000460,
        6099000470, 6099000490, 7699100410, 6099000830, 6099000810, 6002030000, 6002040000, 6002030900, 6002040900, 6099000440, 6099000450, 6099000203, 6099000615, 
        6002140000, 6002140099, 6099000616, 6002150000, 7610001900, 7610001000, 7610001099, 2301010100, 2301010108, 2301010110, 2301010118, 2301010200, 2301010208,
        2301010210, 2301010218, 2301010300, 2301010308, 2301010400, 2301010408, 2301011100, 2301011108, 2301011110, 2301011118, 2301011200, 2301011208, 2301011210,
        2301011218, 2301011300, 2301011308, 2302010000, 2302010007, 1611000000, 1613000000, 1615000000, 7610007900, 7610007000, 7610005000, 7699100620, 7699100420)
        THEN 'Advances to suppliers'
        WHEN "glAccount" IN (6002010000, 6002010099, 6002010900, 6099000612, 6099000211, 2301020100, 2301020108, 2301020110, 2301020118, 2301020200, 2301020208,
        2301020210, 2301020218, 2301020300, 2301020308, 2301020310, 2301020318, 2301021100, 2301021108, 2301021110, 2301021118, 2301021200, 2301021208, 2301021210,
        2301021218, 2301021300, 2301021308, 2301021310, 2301021318, 2301000008, 2301001008, 2302020000, 2302020007, 1612000000, 1614000000)
        THEN 'Capital advances'
        WHEN "glAccount" IN (6001089000, 7625003000, 7610003000, 7615003000, 7699100550, 6099000710, 7699200500, 7615004000, 7629009000, 7625013000, 7625013900,
        7699100200, 7629000900, 7610406000, 7610406900, 7610405000, 7610405900, 7630707000, 7622020000, 7620801000, 7620801900, 7630705000, 7629000099, 7629000000,
        7610004000, 7618000900, 7618000000, 6001080900, 6001080000, 7622010000, 7620703000, 7620704000, 7620701000, 7620701900, 7620702000, 7622030000, 7622040000,
        7699900000, 7610500000, 7610500099, 7699100240, 7630702000, 7630703000, 7630706000, 7625006000, 7625007000, 7625001000, 7625001900, 7620200000, 7620200900,
        2601030000, 2601030008, 2601040000, 2601040008, 2601050009, 2602000000, 2602000008, 2602000009, 2602000010, 2602000018, 2602000019, 2602001000, 2602001008,
        2602001010, 2602001018, 2602010000, 2602010009, 2602060100, 2602000090, 2603000000, 2603000100, 36307060, 1284000000, 1284000900, 1630000000, 1282000000, 
        1282000900, 1251000000, 2601010009, 2601020009)
        THEN 'Other receivables'
        WHEN "glAccount" IN (2201910000, 1290000000, 2190000000, 2301910100, 2602900000)
        THEN 'Provisions'
        WHEN ("glAccount" IN (1022200000))
        THEN 'FX Spot'
        WHEN ("glAccount" IN (1110009000, 1270000000, 1271000000, 2204000000, 2604010100, 2604020100, 2604021110, 2604021200, 7621156000))
        THEN 'Loans issued'
        WHEN ("glAccount" IN (1410000000, 1430010000, 1430030000, 1430050000, 1430060000, 1430070000))
        THEN 'Tax'
        WHEN ("glAccount" IN (2602000020))
        THEN 'MTM Derivatives'        
        WHEN ("glAccount" IN (7610300000))
        THEN 'Dividends'        
        WHEN ("glAccount" IN (7611102000))
        THEN 'Capital expenses'       
        ELSE 'Unknown'
    END AS "accountPurpose"
from RISKACCESS."unifiedARReport"
where 
    ("sourceSystem" = 'SAP ERP' or "sourceSystem" = 'SAP ECP') and
    "rowType" = 'M' and
    "reportType" = 'AR' and 
    "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD')  
union all select
 "customerName","internalExternalEN","orgName", "companyCode", "dueDate", "presentAmountUSD", "dealNumber", "holding", "currencyCode", "sourceSystem",
CASE
        WHEN "glAccount" like '63%'
        THEN 'Provisions'
        WHEN ("glAccount" IN (5201010100, 5201010110, 5201010200, 5201011100, 5201011110, 5201011200))
        THEN 'Trade payables'
        WHEN ("glAccount" IN (5501000000, 5501010000, 5502000000, 5503020000, 5503030000, 5503030010, 5503010000))
        THEN 'Other payables'
        WHEN ("glAccount" like '62%' or "glAccount" IN (5301010000, 5301010010, 7620002000, 0035100000))
        THEN 'Advances received'
        WHEN ("glAccount" like '60%' or "glAccount" IN (5101010100, 5101020110, 5101021110, 5104040000, 5109000000, 5503050000, 5505010200, 5505010210, 5505011210, 5505020100, 
        7610101000, 7610402000, 7610404000, 7610405000, 7610601000, 7610603000, 7610691000, 7619000000, 7620002000, 0033970000, 0033500000, 0033600000, 0033960000, 0033140000,
        0033130000, 0033120000, 0033110000, 0033800000, 0033950000, 0033970100))
        THEN 'Other payables'
        WHEN ("glAccount" IN (0032200000, 0032310000))
        THEN 'Pension'
        WHEN ("glAccount" IN (0032300000, 0032150000, 0032140000, 0032130000, 0032120000, 0032110000, 0031500000, 0033970200))
        THEN 'Social'
        WHEN ("glAccount" IN (0031940000, 0031930000, 0031920000, 0031910000, 0031800000, 0031700000, 0031600000, 0031330000, 0031320000, 0031220000, 0031210000, 0031100000))
        THEN 'Tax'
        WHEN ("glAccount" IN (0030200000))
        THEN 'Loans received'
        ELSE 'Unknown'
    END AS "accountPurpose"
from RISKACCESS."unifiedARReport"
where 
    ("sourceSystem" = 'SAP ERP' or "sourceSystem" = 'SAP ECP') and
    "rowType" = 'M' and
    "reportType" = 'AP' and 
    "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD')  
    '''

data_export = export_from_RISKCUSTOM(query)    

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [3]:
# проверка на кол-во баз данных
assert len(data_export.sourceSystem.unique()) == 5

In [117]:
data_work = data_export.query('holding != "EXTERNAL"').reset_index(drop=True)
data_work = data_work[data_work.accountPurpose.isin(['Trade receivables', 'Other receivables', 'Provisions','Trade payables', 'Other payables'])]
# фильтрация по перемнной Group
data_work.currencyCode = data_work.currencyCode.replace(CCY_tech_dict) # tech ccy to normal

data_work = data_work[~(data_work.orgName.isna() & data_work.companyCode.isna())].reset_index(drop=True)
data_work['org_code'] = merge_Mapping(data_work, col='orgName')
data_work['CompCode_concat'] = data_work['companyCode'].fillna('')
data_work.loc[data_work.CompCode_concat == '','CompCode_concat'] = data_work.loc[data_work.CompCode_concat == '','org_code']
data_work['CompMap'] = merge_SalesUnits(data_work, col='CompCode_concat', merge_col='ocpSegment')

data_work['CtyCode'] = merge_Mapping(data_work, col='customerName').fillna('External')
data_work['CtyMap'] = merge_SalesUnits(data_work, col='CtyCode', merge_col='ocpSegment').fillna('External')
data_work['Cty_holding'] = merge_SalesUnits(data_work,'CtyCode',merge_col='holding').fillna('External')

data_work['Amount_USD'] = data_work['presentAmountUSD'].replace({np.NaN : 0})

data_work_pivot = data_work.pivot_table(index=['holding','customerName', 'Cty_holding', 'CompCode_concat', 'currencyCode', 'sourceSystem','accountPurpose', 'CompMap', 'CtyCode', 'CtyMap','internalExternalEN','dueDate'],values=['Amount_USD'],aggfunc=sum).reset_index()

data_work_pivot['AR_flag'] = data_work_pivot['accountPurpose']\
                                            .isin(['Trade receivables', 'Other receivables', 'Provisions'])\
                                            .astype(int)
data_work_pivot['AP_flag'] = data_work_pivot['accountPurpose']\
                                            .isin(['Trade payables', 'Other payables'])\
                                            .astype(int)

data_work_pivot['Amount_AR_USD'] = data_work_pivot['Amount_USD'] * data_work_pivot['AR_flag']
data_work_pivot['Amount_AP_USD'] = data_work_pivot['Amount_USD'] * data_work_pivot['AP_flag']

data_work_pivot = Period(data_work_pivot, day_for_count=Today, col_with_date='dueDate')

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SAMSALES' 'SAMSALES' 'SAMSALES' ... 'SUEK RU' 'SUEK RU' 'SUEK RU']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', f'{id_col}_merge']
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlit

Проверка на merge:

In [104]:
check_data = data_work.loc[(data_work['internalExternalEN'] == 'Internal') & (data_work['CtyMap'] == 'External'),\
                                                    ['customerName', 'CtyCode', 'CtyMap']].drop_duplicates()
check_data2 = data_work.loc[(data_work['internalExternalEN'] == 'Internal') & (data_work['CompMap'] == 'External'),\
                                                    ['orgName', 'CompCode_concat','CompMap']].drop_duplicates()
check_data3 = data_work.loc[(data_work['CtyCode'] != 'External') & (data_work['Cty_holding'] == 'External'),\
                                                    ['CtyCode', 'Cty_holding']].drop_duplicates()
if print_manual_map == True:
    writer = pd.ExcelWriter('manual_map.xlsx', engine='openpyxl')  
    workbook=writer.book
    startcol = 0
    for check_data in [check_data2, check_data, check_data3]:
        check_data.to_excel(writer, index=False, startcol=startcol)
        startcol = startcol + check_data.shape[1] + 1
    writer.close()


In [28]:
BS_data = data_work_pivot[(data_work_pivot.accountPurpose.str.contains('Trade')) | (data_work_pivot.accountPurpose.str.contains('Other'))].reset_index(drop=True)
BS_data['Comp_hold'] = merge_SalesUnits(BS_data, col='CompCode_concat', merge_col='holding')
BS_data['Cpty_hold'] = merge_SalesUnits(BS_data, col='CtyCode', merge_col='holding')

for Group in ['SUEK', 'EUROCHEM']:
    BS_data_group = BS_data[(BS_data.Comp_hold == Group) & (BS_data.Cpty_hold == Group)]
    BS_data_group = BS_data_group[BS_data_group.CompMap != 'External']
    BS_data_group = BS_data_group[['CompMap', 'CtyMap', 'Amount_USD']]
    BS_pivot = pd.pivot_table(BS_data_group, values='Amount_USD', index='CompMap', aggfunc='sum', columns='CtyMap',)
    
    if Group == 'EUROCHEM':
        BS_pivot_EURO = BS_pivot
    if Group == 'SUEK':
        BS_pivot_SUEK = BS_pivot

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['EUROCHEM' 'EUROCHEM' 'EUROCHEM' ... 'SUEK' 'SUEK' 'SUEK']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', f'{id_col}_merge']
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2

Запись в output файлы:

In [6]:
Output_SUEK = "_".join([str(date.today()), 'SUEK', 'SAP', 'ARAP', f'{Mounth}.xlsx'])
Output_EURO = "_".join([str(date.today()), 'EUROCHEM', 'SAP', 'ARAP', f'{Mounth}.xlsx'])
Sheet_in_output_file = 'ARAP'
if excel_tofolder_on_Z == True:
    Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
    Output_SUEK = Output_path + Output_SUEK
    Output_EURO = Output_path + Output_EURO

if Print_ARAP == True:
    data_print_SUEK = data_work_pivot.query('holding == "SUEK"')[['customerName', 'CompCode_concat', 'currencyCode', 'sourceSystem',\
                        'accountPurpose', 'CompMap', 'CtyCode', 'CtyMap','Amount_USD',\
                        'AR_flag', 'Amount_AR_USD', 'AP_flag', 'Amount_AP_USD', 'Days', 'Period']]
    BS_pivot_SUEK.to_excel(Output_SUEK, sheet_name='BS_to_BS')
    new_list(data_print_SUEK, Output_SUEK, Sheet_in_output_file)
    data_print_EURO = data_work_pivot.query('holding == "EUROCHEM"')[['customerName', 'CompCode_concat', 'currencyCode', 'sourceSystem',\
                        'accountPurpose', 'CompMap', 'CtyCode', 'CtyMap','Amount_USD',\
                        'AR_flag', 'Amount_AR_USD', 'AP_flag', 'Amount_AP_USD', 'Days', 'Period']]
    BS_pivot_EURO.to_excel(Output_EURO, sheet_name='BS_to_BS')
    new_list(data_print_EURO, Output_EURO, Sheet_in_output_file)

In [7]:
assert 1==2

AssertionError: 

Состав ARAP

In [ ]:
data_pivot = data_work[~((data_work.AR_flag==0) & (data_work.AP_flag==0))]
group = ""
for group in ['EUROCHEM', 'SUEK']:
    data_pivot_group = data_pivot[data_pivot.holding == group]
    data_pivot_group = data_pivot_group.pivot_table(index=['holding', 'CompMap', 'CompCode_concat', 'internalExternalEN', 'AR_flag', 'AP_flag'], values='Amount_USD', aggfunc={'Amount_USD':['sum','count']}).reset_index()
    data_pivot_group.to_excel(f'{str(date.today())}_ARAP_pivot_{group}_{Mounth}.xlsx')

Поиск встречных требований:

In [118]:
# Фильт по Counterparty_Group - только внутригрупповые
data_E_to_C = data_work_pivot[(data_work_pivot.CtyMap!='External')&(data_work_pivot.Amount_USD!=0)&(data_work_pivot.accountPurpose!='Provisions')]
data_E_to_C = data_E_to_C[(data_E_to_C.AR_flag!=0)|(data_E_to_C.AP_flag!=0)]
data_E_to_C['Index_copy'] = data_E_to_C.index
data_E_to_C= data_E_to_C.fillna('_')

group_data = data_E_to_C.pivot_table(index=['CompMap', 'CompCode_concat', 'CtyMap', 'CtyCode', 'currencyCode', 'AR_flag', 'AP_flag'], values=['Amount_USD', 'Index_copy'], aggfunc={'Amount_USD':sum, 'Index_copy':list}).reset_index()
group_data['concat_col_EC'] = group_data[['CompCode_concat', 'CtyCode', 'currencyCode', 'AR_flag']].astype(str).agg('.'.join, axis=1)
group_data['concat_col_CE'] = group_data[['CtyCode', 'CompCode_concat', 'currencyCode', 'AP_flag']].astype(str).agg('.'.join, axis=1)
group_data_merge = group_data.merge(group_data, left_on='concat_col_EC', right_on='concat_col_CE', how='left')
group_data_merge = group_data_merge[group_data_merge.CompCode_concat_x==group_data_merge.CtyCode_x] # контрагент равен компании
group_data_merge = group_data_merge[['CompMap_x','CompCode_concat_x','CtyMap_x','CtyCode_x','currencyCode_x','AR_flag_x','AP_flag_x','Amount_USD_x','Index_copy_x','AR_flag_y','AP_flag_y','Amount_USD_y','Index_copy_y']]
group_data_merge = group_data_merge.fillna(0)
group_data_merge[['Amount_USD_x','Amount_USD_y']] = group_data_merge[['Amount_USD_x','Amount_USD_y']].astype(int)
# Сравнение на равенство
not_equal_data = group_data_merge[group_data_merge.Amount_USD_x.abs()!=group_data_merge.Amount_USD_y.abs()]
# To print
Index_copy_x_list = list(set(np.concatenate(not_equal_data.Index_copy_x.values.tolist())))
# Index_copy_y_list = list(set(np.concatenate(not_equal_data[not_equal_data.Index_copy_y!=0].Index_copy_y.values.tolist())))
# index_copy_list = Index_copy_x_list+Index_copy_y_list
# not_equal_data_strings = data_work.loc[index_copy_list,:]
not_equal_data_strings = data_work.loc[Index_copy_x_list,:] # контрагент равен компании
# to excel
not_equal_data = not_equal_data.rename(columns={'CompMap_x':'Segment','CompCode_concat_x':'Code','CtyMap_x':'Segmet_cpty','CtyCode_x':'Code_cty','currencyCode_x':'CCY'})

### ccy list
group_data_ccy = data_E_to_C.pivot_table(index=['currencyCode', 'AR_flag', 'AP_flag'], values=['Amount_USD', 'Index_copy'], aggfunc={'Amount_USD':sum, 'Index_copy':list}).reset_index()
group_data_ccy['concat_col_EC'] = group_data_ccy[['currencyCode', 'AR_flag']].astype(str).agg('.'.join, axis=1)
group_data_ccy['concat_col_CE'] = group_data_ccy[['currencyCode', 'AP_flag']].astype(str).agg('.'.join, axis=1)
group_data_merge_ccy = group_data_ccy.merge(group_data_ccy, left_on='concat_col_EC', right_on='concat_col_CE', how='left')
group_data_merge_ccy = group_data_merge_ccy[['currencyCode_x','AR_flag_x','AP_flag_x','Amount_USD_x','Index_copy_x','AR_flag_y','AP_flag_y','Amount_USD_y','Index_copy_y']]
group_data_merge_ccy = group_data_merge_ccy.fillna(0)
group_data_merge_ccy[['Amount_USD_x','Amount_USD_y']] = group_data_merge_ccy[['Amount_USD_x','Amount_USD_y']].astype(int)
# Сравнение на равенство
not_equal_data_ccy = group_data_merge_ccy[group_data_merge_ccy.Amount_USD_x.abs()!=group_data_merge_ccy.Amount_USD_y.abs()]

if print_counterclaims == True:
    Output_counterclaims = f'{str(date.today())}_ARAP_not_equal_counterclaims.xlsx'
    not_equal_data.to_excel(Output_counterclaims, sheet_name='pivot')
    new_list(not_equal_data_strings, Output_counterclaims, 'data', True)
    new_list(not_equal_data_ccy, Output_counterclaims, 'pivot_ccy')

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12744\640719962.py:7: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_data = data_E_to_C.pivot_table(index=['CompMap', 'CompCode_concat', 'CtyMap', 'CtyCode', 'currencyCode', 'AR_flag', 'AP_flag'], values=['Amount_USD', 'Index_copy'], aggfunc={'Amount_USD':sum, 'Index_copy':list}).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12744\640719962.py:27: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_data_ccy = data_E_to_C.pivot_table(index=['currencyCode', 'AR_flag', 'AP_flag'], values=['Amount_USD', 'Index_copy'], aggfunc={'Amount_USD':sum, 'Ind

In [134]:
if print_counterclaims == True:
    writer = pd.ExcelWriter(Output_counterclaims, engine='openpyxl')  
    workbook=writer.book

# part of the data by holding
holding = ''
for holding in data_E_to_C.holding.unique().tolist():
    data_E_to_C_holding = data_E_to_C[(data_E_to_C.holding == holding) & (data_E_to_C.Cty_holding == holding)]
    startrow = 1
    currencyCode = ''
    if print_counterclaims == True:
        pd.DataFrame({'holding':f'{holding}_by_CCY'}, index=[1]).to_excel(writer, sheet_name=f'{holding}_CCY_tabels', index=False, header=False)

    # matrix for each holding
    CompMap_list = list(data_E_to_C_holding.CompMap.unique())
    CompMap_list.sort(reverse=True)
    CtyMap_list = list(data_E_to_C_holding.CtyMap.unique())
    CtyMap_list.sort(reverse=True)
    matrix_list = []
    for CompMap in CompMap_list:
        matrix_dict = {}
        matrix_dict['CompMap'] = CompMap
        for CtyMap in CtyMap_list:
            matrix_dict[CtyMap] = 0
        matrix_list.append(matrix_dict)
    matrix_df = pd.DataFrame(matrix_list)
    matrix_df.index = matrix_df['CompMap']
    matrix_df = matrix_df.drop('CompMap',axis=1)

    # part of the data by CCY
    for currencyCode in data_E_to_C_holding.currencyCode.unique().tolist():
        data_E_to_C_currencyCode = data_E_to_C_holding[data_E_to_C_holding.currencyCode == currencyCode]
        pivot = data_E_to_C_currencyCode.pivot_table(index=['CompMap','CtyMap'],values=['Amount_AR_USD','Amount_AP_USD'],aggfunc=sum).reset_index()
        
        matrix_df[:] = 0
        ccy_df = matrix_df
        for string in CompMap_list:
            status = 'AR'
            for col in CtyMap_list:
                CCY_df_part = pivot[(pivot.CompMap==string)&(pivot.CtyMap==col)]
                if string==col:
                    status = 'AP'
                    try:
                        ccy_df.loc[string, col] = CCY_df_part.Amount_AR_USD.values[0] + CCY_df_part.Amount_AP_USD.values[0]
                    except:
                        pass
                elif status == 'AP':
                    try:
                        ccy_df.loc[string, col] = CCY_df_part.Amount_AP_USD.values[0]
                    except:
                        pass
                elif status == 'AR':
                    try:
                        ccy_df.loc[string, col] = CCY_df_part.Amount_AR_USD.values[0]
                    except:
                        pass

        ### to excel
        if print_counterclaims == True:
            pd.DataFrame([{'CCY':currencyCode}]).to_excel(writer, sheet_name=f'{holding}_CCY_tabels', index=False, header=False, startrow=startrow)
            startrow += 1
            ccy_df.to_excel(writer, sheet_name=f'{holding}_CCY_tabels', index=True, startrow=startrow)
            startrow = startrow + len(ccy_df) + 2
if print_counterclaims == True:
    # data_work_pivot.to_excel(writer, sheet_name=f'data', index=False)
    writer.close()

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12744\2516715862.py:33: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot = data_E_to_C_currencyCode.pivot_table(index=['CompMap','CtyMap'],values=['Amount_AR_USD','Amount_AP_USD'],aggfunc=sum).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12744\2516715862.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '36386.19' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ccy_df.loc[string, col] = CCY_df_part.Amount_AR_USD.values[0] + CCY_df_part.Amount_AP_USD.values[0]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12744\2516715862.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and 